# Iceberg Classifier Challenge

## Import the data

Read the json data using pandas and replace all the "na" values of incident angle with 0.

In [ ]:
import pandas as pd
train_data = pd.read_json("train.json")
train_data.inc_angle = train_data.inc_angle.replace("na",39.26)

## Preprocess the raw data

Preprocess the raw data into numpy arrays for traininig and resize them to 150*150

In [ ]:
import numpy as np
import scipy
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split


band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_1"]])
band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_2"]])
train = np.concatenate([band1[:, :, :, np.newaxis]
                          , band2[:, :, :, np.newaxis]
                         , ((band1+band2)/2)[:, :, :, np.newaxis]], axis=-1)

#train = np.array([scipy.misc.imresize(x,(150,150)) for x in train])
angle_train = np.array(train_data.inc_angle)
train_labels = np.array(train_data["is_iceberg"])

# Split data in two for validation and traininig
train, valid, angle_train, angle_valid, train_label, valid_label = train_test_split(train
                    , angle_train, train_labels, random_state=25, train_size=0.80)

## Create Data generators

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Concatenate, Conv2D, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Dropout, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

gen=ImageDataGenerator(rotation_range=360, 
                       horizontal_flip=True, 
                       vertical_flip=True, 
                       shear_range=0.2, 
                       zoom_range=0.2,
                       width_shift_range=0.2,
                       height_shift_range=0.2)
batch_size = 64
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=121)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=121)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

gen_flow = gen_flow_for_two_inputs(train, angle_train, train_label)


## Create Model

### Inception-ResNet

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

input1 = Input(shape=(150, 150, 3))
base_model = InceptionResNetV2(include_top=True, 
                               weights='imagenet', 
                               input_tensor=input1, 
                               input_shape=(150, 150, 3), 
                               classes=1000)

x = base_model.output
input2 = Input(shape=(1,))
#x = (Concatenate()([x, BatchNormalization(momentum=0)(input2)]))
x = BatchNormalization(momentum=0)(x)
x = Dropout(0.7)(x)
x = Dense(1000, activation = 'relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


# this is the model we will train
model = Model(inputs=[input1,input2], outputs=predictions)

#first: train only the top layers
for layer in base_model.layers[:len(base_model.layers)-20]:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])

### Simple CNN

In [ ]:
from keras.layers import LeakyReLU

input1 = Input(shape=(75, 75, 3))
b1 = BatchNormalization(momentum=0)(input1)
x = Conv2D(64, (5,5), padding='same')(b1)
x = BatchNormalization(momentum=0)(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (5,5), padding='same')(x)
x = BatchNormalization(momentum=0)(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (5,5), padding='same')(x)
x = BatchNormalization(momentum=0)(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization(momentum=0)(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D()(x)
x = Conv2D(512, (3,3), padding='same')(x)
x = BatchNormalization(momentum=0)(x)
x = LeakyReLU(alpha=0.01)(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
input2 = Input(shape=(1,))
#x = Concatenate()([x,input2])
x = BatchNormalization(momentum=0)(x)
x = Dropout(0.4)(x)
x = Dense(1000)(x)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.25)(x)
x = Dense(512)(x)
x = LeakyReLU(alpha=0.01)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input1,input2], outputs=predictions)

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
model.fit_generator(gen_flow, validation_data=([valid, angle_valid], valid_label),
                    steps_per_epoch=len(train) / batch_size, epochs=10)

In [ ]:
model.optimizer.lr=0.0001

In [ ]:
model.save_weights("m.h5")

## Inference

In [ ]:
test_data = pd.read_json("test.json")
# Test data
test_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_data["band_1"]])
test_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_data["band_2"]])
test = np.concatenate([test_band1[:, :, :, np.newaxis]
                          , test_band2[:, :, :, np.newaxis]
                         , ((test_band1+test_band2)/2)[:, :, :, np.newaxis]], axis=-1)

test = np.array([scipy.misc.imresize(x,(150,150)) for x in test])
angle_test = np.array(test_data.inc_angle)

predictions = model.predict([test,angle_test])

df = test_data[['id']].copy()
df['is_iceberg'] = predictions
df.to_csv('predictions3.csv', index = False)
df.sample(3)

In [ ]:
print(predictions.shape)